In [1]:
import pandas as pd
import numpy as np
import json


import os
import subprocess
import threading

import requests

import time
import datetime

import ta

def get_data(url, index, proxy):    
    global results
    global threads
        
    if proxy == None:
        res = requests.get(url, timeout=2)
    else:
        proxies = {
          "http": "http://" + proxy,
          "https": "https://" + proxy,
        }
        res = requests.get(url, proxies=proxies, timeout=2)
        
    results[index] = pd.DataFrame(json.loads(res.text))

def get_df(start_time, proxy=None, total_range=30):
    global threads
    global results
    
    start_time = pd.to_datetime(start_time).tz_localize(None)
    
    if start_time.date() == datetime.datetime.utcnow().date():
        urls = ["https://www.bitmex.com/api/v1/trade?symbol=XBTUSD&count={}&start={}&reverse=false&startTime={}".format(1000, i * 1000, start_time) for i in range(total_range)]
    else:
        urls = ["https://www.bitmex.com/api/v1/trade?symbol=XBTUSD&count={}&start={}&reverse=false&startTime={}&endTime={}".format(1000, i * 1000, start_time, pd.to_datetime(start_time.date() + pd.Timedelta(days=1))) for i in range(total_range)]
    
    threads = [None] * len(urls)
    results = [None] * len(urls)
    
    for i in range(len(threads)):
        threads[i] = threading.Thread(target=get_data, args=(urls[i], i, proxy))
        threads[i].start()
    
    for i in range(len(threads)):
        threads[i].join()

    df = pd.DataFrame()

    for curr_df in results:
        df = df.append(curr_df, ignore_index=True)
                    
    return df

def manual_scrape(scrape_from, sleep=True):
    print("Manual scrape for {}".format(scrape_from))
    proxy_df = pd.read_csv('proxies', sep=':', header=None)
    proxy_df.columns = ['proxy', 'port', 'username', 'password']

    proxy_df['proxy_string'] =  proxy_df['username'] + ":" + proxy_df['password'] + "@" + proxy_df['proxy'] + ":" + proxy_df['port'].astype(str)
    proxy_list = list(proxy_df['proxy_string'])
    at_once = len(proxy_list) + 1
    all_df = pd.DataFrame()
    completed = False
    
    while True:
        start_time = time.time()
        
        for i in range(at_once):
            if i == 0:
                curr_df = get_df(scrape_from)
            else:
                curr_df = get_df(scrape_from, proxy=proxy_list[i-1])
                
            all_df = all_df.append(curr_df, ignore_index=True)
            all_df = all_df.dropna(subset=['timestamp'], how='all')
            
            scrape_from = all_df.iloc[-1]['timestamp']
            print("Got {} data till {}".format(len(curr_df), scrape_from))
            
            if len(curr_df) < 1000:
                completed = True
                break
         
        total_time_taken = time.time() - start_time
        
        to_sleep = int(60 - total_time_taken) + 1
        
        if completed == True:
            break

        if to_sleep > 0:
            if sleep == True:
                print("Sleeping {} seconds".format(to_sleep))
                time.sleep(to_sleep)
        else:
            print("No need to sleep")
            
    
    all_df['timestamp'] = pd.to_datetime(all_df['timestamp'])
    all_df['timestamp'] = all_df['timestamp'].dt.tz_localize(None)
    all_df = all_df.sort_values('timestamp').reset_index(drop=True)
            
    return all_df

def aws_scrape(name):
    print("AWS Scrape for {}".format(name))
    url = "https://s3-eu-west-1.amazonaws.com/public.bitmex.com/data/trade/{}".format(name)
    r = requests.get(url)
    
    with open('temp', 'wb') as f:
        f.write(r.content)
        
    df = pd.read_csv('temp', compression='gzip')
    os.remove('temp')
    aws_df = df[df['symbol'] == 'XBTUSD']
    aws_df['timestamp'] = pd.to_datetime(aws_df['timestamp'], format="%Y-%m-%dD%H:%M:%S.%f")
    aws_df = aws_df.sort_values('timestamp').reset_index(drop=True)
    return aws_df

def get_bitmex_data(start, end, sleep=True):
    all_df = []

    for scrape_date in pd.date_range(start, end):
        if scrape_date.date() == datetime.datetime.utcnow().date() - pd.Timedelta(days=1):
            curr_time = datetime.datetime.utcnow()
            if curr_time.time() > datetime.time(5,41):
                df = aws_scrape(scrape_date.strftime("%Y%m%d.csv.gz"))
            else:
                df = manual_scrape(scrape_date, sleep=sleep)
        elif scrape_date.date() == datetime.datetime.utcnow().date():
            df = manual_scrape(scrape_date,  sleep=sleep)
        else:
            df = aws_scrape(scrape_date.strftime("%Y%m%d.csv.gz"))


        all_df.append(df)
    
    return pd.concat(all_df, axis=0)

In [2]:
#last one not working

In [3]:
def update_trades():
    end = pd.to_datetime(datetime.datetime.utcnow()).date()
    original_start = end - pd.Timedelta(days=20)

    if os.path.isfile('data/trades.csv'):
        start = pd.to_datetime(subprocess.check_output(["tail", "-1", "data/trades.csv"]).decode().split(",")[0])
    else:
        start = original_start


    
    while True:
        try:
            end = pd.to_datetime(datetime.datetime.utcnow())
            print("{} to {}".format(start, end))
            df = get_bitmex_data(start, end)
            df = df[['timestamp', 'symbol', 'side', 'size', 'price', 'homeNotional', 'foreignNotional']]
            
            if not os.path.isfile('data/trades.csv'):
                df.to_csv("data/trades.csv", index=None)
            else:
                df.to_csv("data/trades.csv", index=None, header=None, mode='a')
                
            break
        except:
            print("Exception. Retrying in 5 secs")
            time.sleep(5)

In [77]:
update_trades()

2021-01-18 10:01:47.658000 to 2021-01-18 14:53:33.085428
Manual scrape for 2021-01-18 10:01:47.658000
Got 24012 data till 2021-01-18T11:52:16.489Z
Got 24012 data till 2021-01-18T12:40:14.686Z
Got 24012 data till 2021-01-18T14:31:14.245Z
Got 12461 data till 2021-01-18T14:53:43.094Z
Got 40 data till 2021-01-18T14:53:46.200Z


In [4]:
def get_significant_traders(df):
    df = df[['timestamp', 'side', 'homeNotional', 'foreignNotional']]
    df = df.groupby(['timestamp', 'side']).sum() 
    df = df.reset_index()
    df = df[df['foreignNotional'] > 500]
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['price'] = df['foreignNotional']/df['homeNotional']
    df = df.sort_values('timestamp')
    df = df.drop_duplicates()
    return df

def get_features(curr_df):
    ser = {}
    curr_df = curr_df.sort_values('timestamp')
    
    if len(curr_df) > 0:
        ser['open'] = curr_df.iloc[0]['price']
        ser['high'] = curr_df['price'].max()
        ser['low'] = curr_df['price'].min()
        ser['close'] = curr_df.iloc[-1]['price']
        ser['volume'] = curr_df['foreignNotional'].sum()
    else:
        ser['open'] = np.nan
        ser['high'] = np.nan
        ser['low'] = np.nan
        ser['close'] = np.nan
        ser['volume'] = np.nan
        
    buy_orders = curr_df[curr_df['side'] == 'Buy']
    sell_orders = curr_df[curr_df['side'] == 'Sell']

    total_buy = buy_orders['homeNotional'].sum()
    total_sell = sell_orders['homeNotional'].sum()
    total = total_buy + total_sell

    ser['buy_percentage'] = total_buy/total
    ser['buy_volume'] = total_buy
    ser['all_volume'] = total
    
    readable_bins = []
    

    readable_bins = [0, 2, 10, np.inf]
        
    readable_labels = ['small', 'medium', 'large']
    curr_df['new_range'] = pd.cut(curr_df['homeNotional'], readable_bins, include_lowest=True, labels=readable_labels).astype(str)
    
        
    for curr_range in set(readable_labels):
        group = curr_df[curr_df['new_range'] == curr_range]
        ser["percentage_{}".format(curr_range)] = group['homeNotional'].sum()/total
        buy_orders = group[group['side'] == 'Buy']
        ser['buy_percentage_{}'.format(curr_range)] = (buy_orders['homeNotional'].sum())/group['homeNotional'].sum()

    
        
    return pd.Series(ser)

def get_features_from_sig(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    minute_only = df['timestamp'].dt.minute.astype(str)
    minute_only_two = minute_only.apply(lambda x: str(x)[1:]) #there is a mistake here.
    df = df[~((minute_only == '9') | (minute_only_two == '9') | (minute_only == '8')  | (minute_only_two == '8'))]

    features = df.groupby(pd.Grouper(key='timestamp', freq="10Min", label='left')).apply(get_features)
    features = features.reset_index()

    features['timestamp'] = pd.to_datetime(features['timestamp'])
    features = features.drop_duplicates(subset=['timestamp'])
    features = features.sort_values('timestamp')
    features = features.dropna()
    return features

In [15]:
df = pd.read_csv('data/trades.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
last_date = df['timestamp'].iloc[-1]

#remove after last full
time_df = pd.DataFrame(pd.Series({'Time': last_date})).T
prev_date = time_df.groupby(pd.Grouper(key='Time', freq="10Min", label='left')).sum().index[0]

if (last_date.time().minute <= 8):
    prev_date = prev_date - pd.Timedelta(minutes=10)

have_till = prev_date

df = df[df['timestamp'] < have_till]


In [17]:
#remove already calculated
startTime = df.iloc[0]['timestamp']

time_df = pd.DataFrame(pd.Series({'Time': startTime})).T
startTime = time_df.groupby(pd.Grouper(key='Time', freq="10Min", label='left')).sum().index[0]



if os.path.isfile('data/features.csv'):
    startTime = pd.to_datetime(pd.read_csv('data/features.csv').iloc[-1]['timestamp']) + pd.Timedelta(minutes=10)

df = df[df['timestamp'] >= startTime]


In [18]:
#calculate and save features
df = get_significant_traders(df)
features = get_features_from_sig(df)

features['change'] = ((features['close'] - features['open'])/features['open']) * 100
features = features[['timestamp', 'open', 'high', 'low', 'close', 'volume', 'change', 'percentage_large', 'buy_percentage_large']]


<ipython-input-4-990a85e2f421>:53: RuntimeWarning: invalid value encountered in double_scalars
  ser['buy_percentage_{}'.format(curr_range)] = (buy_orders['homeNotional'].sum())/group['homeNotional'].sum()
<ipython-input-4-990a85e2f421>:53: RuntimeWarning: invalid value encountered in double_scalars
  ser['buy_percentage_{}'.format(curr_range)] = (buy_orders['homeNotional'].sum())/group['homeNotional'].sum()
<ipython-input-4-990a85e2f421>:53: RuntimeWarning: invalid value encountered in double_scalars
  ser['buy_percentage_{}'.format(curr_range)] = (buy_orders['homeNotional'].sum())/group['homeNotional'].sum()
<ipython-input-4-990a85e2f421>:53: RuntimeWarning: invalid value encountered in double_scalars
  ser['buy_percentage_{}'.format(curr_range)] = (buy_orders['homeNotional'].sum())/group['homeNotional'].sum()


In [19]:
features

,timestamp,open,high,low,close,volume,change,percentage_large,buy_percentage_large
1,2021-01-18 10:00:00,36563.071298,36630.036630,36390.101892,36403.349108,14107643.0,-4.368402e-01,0.121789,0.290583
2,2021-01-18 10:10:00,36403.349108,36416.493248,36245.016310,36297.640653,13609587.0,-2.903811e-01,0.146924,0.000000
3,2021-01-18 10:20:00,36218.761318,36245.016310,36153.289949,36231.884058,9909611.0,3.623188e-02,0.169879,0.297140
4,2021-01-18 10:30:00,36284.470247,36297.640653,36179.450072,36284.470247,10620016.0,2.005254e-14,0.251910,0.483730
6,2021-01-18 10:50:00,36271.309394,36297.640653,36179.450072,36205.648081,12556831.0,-1.810282e-01,0.468698,0.266544
7,2021-01-18 11:00:00,36231.884058,36271.309394,35855.145213,35906.642729,31478353.0,-8.976661e-01,0.272385,0.000000
8,2021-01-18 11:10:00,35868.005739,36023.054755,35842.293907,36023.054755,10263031.0,4.322767e-01,0.035089,1.000000
9,2021-01-18 11:20:00,35932.447000,35997.120230,35906.642729,35997.120230,1182829.0,1.799856e-01,0.396289,1.000000
11,2021-01-18 11:40:00,35984.166967,36010.082823,35842.293907,35868.005739,14621234.0,-3.228121e-01,0.297829,0.724184
12,2021-01-18 11:50:00,36166.365280,36523.009496,36166.365280,36416.605972,28565904.0,6.919155e-01,0.136842,0.329393


In [20]:
if os.path.isfile('data/features.csv'):
    old_features = pd.read_csv('data/features.csv')
    old_features['timestamp'] = pd.to_datetime(old_features['timestamp'])
    features = pd.concat([old_features, features])
    
features['macd'] = ta.trend.macd_signal(features['close'])
features['rsi'] = ta.momentum.rsi(features['close'])

In [21]:
features

,timestamp,open,high,low,close,volume,change,percentage_large,buy_percentage_large,macd,rsi
0,2020-12-29 00:00:00,27039.601111,27070.750337,26917.900404,26997.840173,17855664.0,-0.154444,0.287782,0.749656,NaN,100.000000
1,2020-12-29 00:10:00,27027.027027,27027.027027,26888.939570,26910.656620,8971637.0,-0.430571,0.261058,0.000000,NaN,0.000000
2,2020-12-29 00:20:00,26917.904133,26939.662930,26874.488771,26874.488771,5394791.0,-0.161288,0.203961,0.636613,NaN,0.000000
3,2020-12-29 00:30:00,26881.720430,26881.720430,26759.431718,26816.842414,11245571.0,-0.241346,0.176560,0.372606,NaN,0.000000
4,2020-12-29 00:40:00,26824.034335,26860.064864,26737.967914,26816.841216,9375515.0,-0.026816,0.201408,0.532002,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
24,2021-01-18 13:50:00,36927.621861,37023.324695,36927.621861,36927.621861,5504816.0,0.000000,0.181461,1.000000,257.211935,58.779846
25,2021-01-18 14:00:00,36845.983788,36941.263391,36832.412523,36873.156342,9926842.0,0.073746,0.193158,0.776994,249.744037,56.924116
27,2021-01-18 14:20:00,36873.156342,36973.190012,36873.156342,36913.990402,5319228.0,0.110742,0.246024,1.000000,240.955975,57.994833
28,2021-01-18 14:30:00,36886.757654,36913.990402,36089.549026,36284.470247,79069371.0,-1.632801,0.360246,0.209933,221.208332,41.053092


In [22]:

if os.path.isfile('data/features.csv'):
    features.to_csv('data/features.csv', header=None, mode='a', index=None)
else:
    features.to_csv('data/features.csv', index=None)




In [9]:
#run update_trade every 7th minute and at start

In [11]:
#after done, run update trade again and backtest verify